# Task 1
Нужно написать две программы: Первая генерирует бинарный файл (min 2Гб), состоящий из случайных 32-рязрядных беззнаковых целых чисел (big endian). Вторая считает сумму этих чисел (с применением длинной арифметики), находит минимальное и максимальное число.

Реализуйте две версии - 1. Простое последовательное чтение 2. Многопоточная + memory-mapped files. Сравните время работы.

## Генерация данных
В файле с заданными рамером и названием генерируется массив беззнаковых чисел с байтовым порядком Big Endian.

In [1]:
from generator import generate

In [2]:
filename = 'generated_numbers_32b'
size = 2048*1024*1024

In [3]:
generate(size, filename)

## Последовательное чтение NumPy
Последовательное считывание данных и нахождение с использованием numpy суммы, минимального и максимального числа.

In [4]:
import numpy as np
from generator import read_f

from generator import disp_res

In [6]:
%%time
array = read_f(filename)
disp_res(np.min(array), np.max(array), np.sum(array))

Min:  13
Max:  4294967292
Sum:  1420583910
Wall time: 2.73 s


## Последовательное чтение Python
Последовательное считывание данных и нахождение стандартными способами суммы, минимального и максимального чисел.

In [7]:
%%time
array = read_f(filename)
disp_res(min(array), max(array), sum(array))

Min:  13
Max:  4294967292
Sum:  1152947155572122598
Wall time: 3min 5s


## Паралельное чтение multiprocessing потоки + numpy
Для чтения используются memory map files. Для распараллеливания используется пул потоков 8 ядер в соответствии с процессором (Intel Core i7-10875H)

In [14]:
from generator import read_mmap_f
from multiprocessing.dummy import Pool
from toolz import juxt

num_workers

8

In [16]:
%%time
read_file = read_mmap_f(size, filename)
splited_arrays = np.array_split(read_file, num_workers)
result = np.array(Pool(num_workers).map(juxt(np.sum, np.min, np.max), splited_arrays))

disp_res(np.min(result[:, 1]), np.max(result[:, 2]), np.sum(result[:, 0]))

Min:  13
Max:  4294967292
Sum:  1420583910
Wall time: 535 ms


## Паралельное чтение multiprocessing потоки + стандартные функции python
Для чтения используются memory map files. Для распараллеливания используется пул потоков 8 ядер в соответствии с процессором (Intel Core i7-10875H)

In [10]:
%%time
read_file = read_mmap_f(size, filename)
splited_arrays = np.array_split(read_file, num_workers)
result = np.array(Pool(num_workers).map(juxt(sum, min, max), splited_arrays))

disp_res(min(result[:, 1]), max(result[:, 2]), sum(result[:, 0]))

Min:  13
Max:  4294967292
Sum:  1152947155572122598
Wall time: 2min 59s


## Паралельное чтение multiprocessing процессы + numpy
Для чтения используются memory map files. Для распараллеливания – пул процессов.

In [11]:
from multiprocessing import Pool as pool_proc

In [13]:
%%time
read_file = read_mmap_f(size, filename)
splited_arrays = np.array_split(read_file, num_workers)
result = np.array(pool_proc(num_workers).map(juxt(np.sum, np.min, np.max), splited_arrays))

disp_res(np.min(result[:, 1]), np.max(result[:, 2]), np.sum(result[:, 0]))

Min:  13
Max:  4294967292
Sum:  1420583910
Wall time: 6.61 s


# Итоги работы
Можно заметить, что релуьтаты улучшаяются в разы по скорости при использовании библиотеки numpy.
В итоге лучший результат в 535мс был получен при использовании распараллеливания на основе потоков и вместе с numpy.
Кроме того, были произведены замеры скорости по вычислению и с помощью потоков, и с помощью процессов. В первом случае результат составил 535мс, во втором – 6.61с. Но в любом случае это намного лучше, чем аналогичные процедуры с последовательным чтением и без memory-mapped files: 2.73с и 3м5с соответственно.